In [15]:

import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import sys, os
sys.path.append(os.path.abspath('..'))
from environment.grid import GridEnv, Action
from enum import Enum


%load_ext autoreload
%autoreload 2
from agent_deepqn.agent import DeepQAgent

from utilities.helper import mean, flatten


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
env = GridEnv(1, agents_start=[], goals_start=[(1,1)])
env.seed(0)
print('Number of actions: ', env.action_space.n)
print('State space: ', env.state_space)


Not all _agents_ have fixed starting positions, rest (1) will be random

Final Self players:  [(2, 4)]
Final Self goals:  [(1, 1)]
Number of actions:  4
State space:  4


In [17]:
samp_frames = 1
agent = DeepQAgent(state_size=env.state_space, action_size=env.action_space.n, seed=0, samp_frames=samp_frames)

In [18]:
# watch an untrained agent
state = [item for sublist in env.reset() for item in sublist]
print(state)
for j in range(200):
    action = agent.act(state)
    states, rewards, done = env.step([action])
    #state, reward = states[0], rewards[0]
    if done:
        break 
        
env.close()

[0.75, 0.75, 0.125, 0.125]


In [19]:


def dqn(agents, num_agents = 2, n_episodes=10000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = [[] for _ in np.arange(num_agents)]                       # list containing scores from each episode
    scores_window = [deque(maxlen=100)  for _ in np.arange(num_agents)] # last 100 scores for each agent
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        states = env.reset()
        prev_states=states.copy()
        eps_scores = [0 for _ in np.arange(num_agents)]
        for t in range(max_t):
            actions = [agent.act(flatten([flatten(states),flatten(prev_states)]), eps) for agent in agents]
            prev_state = states.copy()
            next_states, rewards, done = env.step(actions)

            [agent.step(flatten(states), action, reward, flatten(next_states), done) for agent, action, reward in zip(agents, actions, rewards)]
            states = next_states
            eps_scores = [score + reward for score, reward in zip(eps_scores, rewards)]
            if done:
                break 

        [scores_window[index].append(eps_scores[index]) for index, _ in enumerate(agents)]       # save most recent score
        [scores[index].append(eps_scores[index]) for index, _ in enumerate(agents)]       # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
    return scores

In [20]:
num_agents = 1
samp_frames = 2
agents = [DeepQAgent(state_size=env.state_space, action_size=env.action_space.n, seed=0, samp_frames = samp_frames) for _ in np.arange(num_agents)]

print(agents[0].qnetwork_local)

QNetwork(
  (fc1): Linear(in_features=8, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=4, bias=True)
)


In [21]:
scores = dqn(agents, num_agents, n_episodes=500)

Episode 21	Average Score: -147.52

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
plt.title("Score per episode")
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores[0][:3000])), scores[0][:3000])
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
# load the weights from file
agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

for i in range(3):
    state = env.reset()
    for j in range(200):
        action = agent.act(state)
        env.render()
        state, reward, done, _ = env.step(action)
        if done:
            break 
            
env.close()